In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import math
import sys
print(f'python version: {sys.version}')
print(f'numpy version: {np.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'nibabel version: {nib.__version__}')

def int2seg(intensity):
    if intensity <= 7:
        return f"C{intensity}"
    elif intensity >7 and intensity <= 19:
        return f"T{intensity-7}"
    else:
        return f"L{intensity-19}"
    
df = pd.read_pickle("mendez_df")
df.head()

python version: 3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:49:06) 
[Clang 14.0.6 ]
numpy version: 1.23.5
pandas version: 1.5.3
nibabel version: 5.1.0


Segment  MEAN  STD                        Mesure
0      C2  11.3  1.5                   DREZ length
1      C2  12.6  0.9            Seg.L at dorsal CE
2      C2  13.1  1.0                   Seg.L at BE
3      C2  24.4  0.8  Inf.Art facet to CR distance
4      C2  15.5  4.2             IF to RR distance

In [2]:
PATH = "../Code/spinalcordtoolbox/data/PAM50/template/"

In [3]:
# Import foramen data
foramen_level = nib.load('test_foramen.nii.gz')
x_f,y_f,z_f = np.where(foramen_level.get_fdata()>0)
print(z_f)

[932 731 691 898 765 797 830 865]


In [4]:
# Import Centerline data
center = nib.load(f'{PATH}PAM50_centerline.nii.gz')
x_c, y_c, z_c = np.where(center.get_fdata()>0)

In [5]:
# Test triangle
len_level = {}
for i in range(len(x_f)):
    x = x_f[i]
    y = y_f[i]
    z = z_f[i]
    intensity = int(foramen_level.get_fdata()[x,y,z])
    seg = int2seg(intensity)
    seg_df = df[df.Segment == seg]
    RR = seg_df[seg_df.Mesure == "IF to RR distance"]
    CR = seg_df[seg_df.Mesure == "IF to CR distance"]
    center_val = np.where(z_c == z)
    try :
        x_y_c = [x_c[center_val][0], y_c[center_val][0]]
    except:
        x_y_c = [70, 70]
    dist_c = math.dist([x,y], x_y_c)
    # Because we take the centerline for some case, no estimation based on a triangle can be made, so let the estimation at 0.
    try:
        len_RR = math.sqrt((RR.MEAN*2)**2 - dist_c**2)
    except :
        len_RR = 0
    try:
        len_CR = math.sqrt((CR.MEAN*2)**2 - dist_c**2)
    except:
        len_CR = 0
    
    len_level[intensity] = (z-len_RR, z-len_CR)

In [6]:
# Import Cord data
PATH = f'{PATH}PAM50_cord.nii.gz'
cord = nib.load(PATH)
p50_mask = cord.get_fdata()
p50_mask.shape

(141, 141, 991)

In [7]:
cord_mask = np.copy(p50_mask)
for lvl in len_level:
    level_r = int(len_level[lvl][0])
    level_c = int(len_level[lvl][1])
    cord_mask[:, :, level_r][cord_mask[:, :, level_r]>0] = lvl
    cord_mask[:, :, level_c][cord_mask[:, :, level_c]>0] = lvl
cord_mask[:, :, :][cord_mask[:, :, :]==1] = 0
test_img = nib.Nifti1Image(cord_mask, header=cord.header, affine=cord.affine)
nib.save(test_img, f'test.nii.gz')  # TO CHANGE path output

In [8]:
## No estimation possible for C2/C3, C4 rostral, C5 rostral ...